<a href="https://colab.research.google.com/github/Ddshsp/lab/blob/main/%D0%A8%D0%94%D0%94_%D0%9B%D0%A05_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS via pip!  Please stand by, should be quick...
***********************************************************************



In [ ]:
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/ubuntu-toolchain-r/test/ubuntu/ jammy main'
Description:
Toolchain test builds; see https://wiki.ubuntu.com/ToolChain

More info: https://launchpad.net/~ubuntu-toolchain-r/+archive/ubuntu/test
Adding repository.
Found existing deb entry in /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding deb entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Found existing deb-src entry in /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding key to /etc/apt/trusted.gpg.d/ubuntu-toolchain-r-ubuntu-test.gpg with fingerprint 60C317803A41BA51845E371A1E9377A2BA9EF27F
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 ht

In [ ]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...


In [ ]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

Found existing installation: cffi 1.15.1
Uninstalling cffi-1.15.1:
  Successfully uninstalled cffi-1.15.1
Found existing installation: cryptography 40.0.1
Uninstalling cryptography-40.0.1:
  Successfully uninstalled cryptography-40.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.3/446.3 kB 24.8 MB/s eta 0:00:00
Installing RAPIDS Stable 23.10
Starting the RAPIDS install on Colab.  This will take about 15 minutes.
Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mamba


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca

In [ ]:
import cudf, cuml

In [ ]:
cudf.__version__

'23.10.01'

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
n = 1000000
data = {
    'sex': np.random.choice(['male','female'],size = n),
    'age': np.random.randint(18, 65, size=n),
    'income': np.random.randint(20000, 100000, size=n),
    'profession': np.random.choice(['manager','worker','unemployed','housewife'], size=n)
}

In [ ]:
#%%timeit
df1 = pd.DataFrame(data)

In [ ]:
df1

,sex,age,income,profession
0,male,40,28903,manager
1,male,25,56654,unemployed
2,male,61,60293,housewife
3,female,51,77308,unemployed
4,female,34,36430,manager
...,...,...,...,...
999995,male,44,61783,unemployed
999996,male,27,24617,housewife
999997,female,48,69104,worker
999998,female,64,95628,manager


In [ ]:
%%timeit
df1.to_csv('df1.csv', index=False)


2.17 s ± 651 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
cdf1 = cudf.read_csv('df1.csv')

25.5 ms ± 812 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
cdf2 = cudf.DataFrame.from_pandas(df1)

122 ms ± 19.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# Добавление пропущенных значений
df1.loc[df1.sample(frac=0.1).index, 'age'] = np.nan
df1.loc[df1.sample(frac=0.1).index, 'income'] = np.nan


In [ ]:
df1

,sex,age,income,profession
0,male,NaN,28903.0,manager
1,male,25.0,56654.0,unemployed
2,male,61.0,60293.0,housewife
3,female,NaN,NaN,unemployed
4,female,34.0,36430.0,manager
...,...,...,...,...
999995,male,44.0,61783.0,unemployed
999996,male,NaN,24617.0,housewife
999997,female,NaN,69104.0,worker
999998,female,64.0,NaN,manager


In [ ]:
%%timeit
# Преобразование df в cuDF
cdf2 = cudf.DataFrame.from_pandas(df1)

153 ms ± 2.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
#%%timeit
df1['age'].fillna((df['age'].shift() + df1['age'].shift(-1)) / 2, inplace=True)
df1['income'].fillna((df['income'].shift() + df1['income'].shift(-1)) / 2, inplace=True)


In [ ]:
df1

,sex,age,income,profession
0,male,NaN,28903.0,manager
1,male,25.0,56654.0,unemployed
2,male,61.0,60293.0,housewife
3,female,36.0,51225.5,unemployed
4,female,34.0,36430.0,manager
...,...,...,...,...
999995,male,44.0,61783.0,unemployed
999996,male,54.0,24617.0,housewife
999997,female,54.0,69104.0,worker
999998,female,64.0,NaN,manager


In [ ]:
df1.ffill(inplace=True)

In [ ]:
df1.bfill(inplace=True)

In [ ]:
df1

,sex,age,income,profession
0,male,25.0,28903.0,manager
1,male,25.0,56654.0,unemployed
2,male,61.0,60293.0,housewife
3,female,36.0,51225.5,unemployed
4,female,34.0,36430.0,manager
...,...,...,...,...
999995,male,44.0,61783.0,unemployed
999996,male,54.0,24617.0,housewife
999997,female,54.0,69104.0,worker
999998,female,64.0,69104.0,manager
